In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-intersection/")
CSM = CarlaScenesManager(cpath)
idx = 0
CDM = CSM.get_scene_dataset_by_index(idx)
vid_folder = f"videos_intersection_{idx}"

print(CSM.scenes)
print(f"{len(CDM)} frames")

In [ ]:
from datetime import datetime, timedelta
from avstack.geometry import GlobalOrigin3D
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avstack.modules.tracking.stonesoup import StoneSoupKalmanTracker3DBox

t0 = datetime.now()
percep = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle", gpu=0)
tracker = StoneSoupKalmanTracker3DBox(t0=t0)

agent = 0
sensor = "lidar-0"
agent_frames = CDM.get_frames(sensor=sensor, agent=agent)[1:-1]
n_frames_max = 2000
n_frames = min(n_frames_max, len(agent_frames))
dets_ss_all = set()
ground_truth = set()
ts = []
for frame in tqdm(agent_frames[:n_frames]):
    pc = CDM.get_lidar(frame=frame, sensor=sensor, agent=agent)
    calib = CDM.get_calibration(frame=frame, sensor=sensor, agent=agent)
    objs = CDM.get_objects(frame=frame, sensor=sensor, agent=agent)
    timestamp = tracker.t0 + timedelta(seconds=pc.timestamp)
    ts.append(timestamp)

    # perception
    dets = percep(pc)
    # dets.data = [dets[0]]
    dets.apply("change_reference", GlobalOrigin3D, inplace=True)

    # tracking
    calib.reference = GlobalOrigin3D
    tracks = tracker(dets, platform=GlobalOrigin3D, calibration=calib)
    dets_ss_all.update(tracker._last_detections)

In [ ]:
from stonesoup.plotter import AnimatedPlotterly

plotter = AnimatedPlotterly(ts, tail_length=1)
# plotter.plot_ground_truths(groundtruth, mapping=[0, 2])
plotter.plot_measurements(
    dets_ss_all, mapping=[0, 1], resize=True, convert_measurements=False
)
plotter.plot_tracks(
    tracks.data, mapping=[0, 2], uncertainty=True, resize=True, plot_history=True
)
plotter.show()